In [ ]:
import networkx as nx
from rdkit import Chem
from rdkit.Chem import AllChem, rdmolops
import numpy as np

def add_explicit_hydrogens(smiles):
    # Convert SMILES to RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError("Invalid SMILES string")
    
    # Add explicit hydrogens
    mol_with_h = Chem.AddHs(mol)
    return mol_with_h

def calculate_wiener_index(mol):
    # Get the distance matrix
    dist_matrix = AllChem.GetDistanceMatrix(mol)
    
    # Calculate Wiener Index (sum of upper triangular part of distance matrix)
    wiener_index = np.sum(np.triu(dist_matrix))
    return wiener_index

def calculate_hyper_wiener_index(mol, wiener_index):
    # Get the distance matrix
    dist_matrix = AllChem.GetDistanceMatrix(mol)
    
    # Calculate Hyper-Wiener Index
    num_atoms = mol.GetNumAtoms()
    squared_distance_sum = 0.0
    
    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            distance = dist_matrix[i, j]
            squared_distance_sum += distance ** 2
    
    hyper_wiener_index = wiener_index / 2.0 + squared_distance_sum / 2.0
    return hyper_wiener_index

def calculate_harary_index(mol):
    # Get the distance matrix
    dist_matrix = AllChem.GetDistanceMatrix(mol)
    
    # Calculate Harary Index with fractional terms
    num_atoms = mol.GetNumAtoms()
    harary_index = 0.0
    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            distance = dist_matrix[i, j]
            if distance > 0:  # Avoid division by zero
                harary_index += 1.0 / distance
    
    return harary_index

def calculate_detour_index(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None

    # Ensure hydrogens are considered
    mol = Chem.AddHs(mol)  # Add hydrogens explicitly if not present
    
    # Create an adjacency matrix from the RDKit molecule
    g = rdmolops.GetAdjacencyMatrix(mol)
    
    # Convert to NetworkX graph
    G = nx.from_numpy_array(g)
    
    # Initialize indices
    detour_index = 0
    detour_harary_index = 0.0
    
    # Compute all simple paths between all pairs of nodes
    nodes = list(G.nodes())
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            try:
                all_paths = list(nx.all_simple_paths(G, source=nodes[i], target=nodes[j]))
                if all_paths:
                    # Find the longest path among all simple paths
                    longest_path = max(all_paths, key=len)
                    length = len(longest_path) - 1
                    if length > 0:
                        detour_index += length
                        detour_harary_index += 1 / length
            except nx.NetworkXNoPath:
                continue

    return detour_index, detour_harary_index

# Example SMILES for a molecule
canonical_smiles = ''

# Add explicit hydrogens
mol_with_h = add_explicit_hydrogens(canonical_smiles)

# Calculate Wiener Index
wiener_index = calculate_wiener_index(mol_with_h)
print(f"Wiener Index: {wiener_index:.4f}")

# Calculate Hyper-Wiener Index
hyper_wiener_index = calculate_hyper_wiener_index(mol_with_h, wiener_index)
print(f"Hyper-Wiener Index: {hyper_wiener_index:.4f}")

# Calculate Harary Index
harary_index = calculate_harary_index(mol_with_h)
print(f"Harary Index: {harary_index:.4f}")

# Calculate Detour Index and Detour Harary Index
detour_index, detour_harary_index = calculate_detour_index(canonical_smiles)
print(f"Detour Index: {detour_index:.4f}")
print(f"Detour Harary Index: {detour_harary_index:.4f}")
